In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tf = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


train_dl = DataLoader(
    datasets.ImageFolder('data/Training', tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True
)

test_dl = DataLoader(
    datasets.ImageFolder('data/Testing', tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True
)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128 * 16 * 16, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 4)
).to(device)

In [4]:
opt = optim.AdamW(model.parameters(), 1e-4)
loss_fn = nn.CrossEntropyLoss()

In [7]:
model.train()

for epoch in range(25):
    running_loss = 0

    for x, y in train_dl:
        opt.zero_grad()

        loss = loss_fn(model(x.to(device)), y.to(device))
        loss.backward()
        
        running_loss += loss

        opt.step()

    print(f'Epoch {epoch+1}: Loss was {running_loss}')

Epoch 1: Loss was 153.25660705566406
Epoch 2: Loss was 101.19564819335938
Epoch 3: Loss was 82.53191375732422
Epoch 4: Loss was 72.28308868408203
Epoch 5: Loss was 65.00205993652344
Epoch 6: Loss was 57.7688102722168
Epoch 7: Loss was 50.11883544921875
Epoch 8: Loss was 45.23326873779297
Epoch 9: Loss was 41.178436279296875
Epoch 10: Loss was 34.92885208129883
Epoch 11: Loss was 32.432044982910156
Epoch 12: Loss was 29.602455139160156
Epoch 13: Loss was 25.5443115234375
Epoch 14: Loss was 22.668842315673828
Epoch 15: Loss was 20.021303176879883
Epoch 16: Loss was 19.479610443115234
Epoch 17: Loss was 17.62127685546875
Epoch 18: Loss was 15.198801040649414
Epoch 19: Loss was 14.0827054977417
Epoch 20: Loss was 13.675762176513672
Epoch 21: Loss was 10.190719604492188
Epoch 22: Loss was 10.432578086853027
Epoch 23: Loss was 8.86497974395752
Epoch 24: Loss was 8.283863067626953
Epoch 25: Loss was 9.139556884765625


In [6]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("device:", torch.cuda.get_device_name(0))
    print("cuda runtime:", torch.version.cuda)

torch: 2.8.0+cu129
cuda available: True
device: NVIDIA GeForce RTX 5060 Ti
cuda runtime: 12.9


In [11]:
model.eval()
test_loss, correct = 0.0, 0

with torch.no_grad():
    for x,y in test_dl:
        x, y = x.to(device), y.to(device)

        logits = model(x)
        test_loss += loss_fn(logits, y).item() * y.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()

test_loss /= len(test_dl.dataset)
accuracy = 100.0 * correct / len(test_dl.dataset)

print('Test Loss:', test_loss, 'Test Accuracy:', accuracy, '%')

Test Loss: 0.13237491785672945 Test Accuracy: 96.5675057208238 %
